# CHESS DATA ANALYSIS

TEAM: Abraham Borg, Mehar Rekhi, Sarom Thin, Cristian Vazquez

INTRO: We want to analyze a data set of chess games and use the information to make predictions about who will win a game of chess. The data is taken from this source: https://www.kaggle.com/datasets/milesh1/35-million-chess-games?resource=download 

DESCRIPTION OF SOURCE DATA SET: The data shows all game moves for White and Black. The date of each game is specified. We are also given the results of each game. We know how many moves each player made. Finally, we know what the ELO rating of each player was at the time of the game. There is a lot of information that we are not interested in so that will need to be cleaned up. Also, the list of moves is a long string, but we want to have each move occupy its own dataframe column.

The complete dataset is 3.5 million games (filesize about 2GB), but for this experiment we initially selected the first 30,000 games for analysis. It was clear that this sample was heavily skewed for White. So instead we opted to randomly select 2 million games.   

The data set was loaded into the dataframe from the local machine, and then the pickle of the completely prepared dataset was loaded onto github.

PREDICTION AND FEATURES TO USE AS PREDICTORS: 
PREDICTORS:
- What were the most common first 8 moves for white and black each year? How have these opening moves changed over time?
- For each year in the data set, which pieces were most commonly left in play at the endgame? 
- Is there a piece in the game that is a good predictor of who will win? For example, if white loses a bishop first, does that correlate with more losses for white? 
- If a player still has their queen and the other player doesn't, does the player with the queen always win? 
- Can we build a model that predicts who will win a game based on the first 8 chess moves? 
- Can our model predict who will win based on the endgame pieces available to both players?

PREDICT: Who will win based on the openings for each player?

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
from matplotlib import rcParams
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# allow output to span multiple output lines in the console
pd.set_option('display.max_columns', 500)

# switch to seaborn default stylistic parameters
# see the useful https://seaborn.pydata.org/tutorial/aesthetics.html

sns.set()
sns.set_context('paper') # 'talk' for slightly larger

# change default plot size
rcParams['figure.figsize'] = 9,7

In [2]:
# code in this cell from: 
# https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to display/hide the code."></form>''')

# DATA PREPARATION

The data set is given with many columns that we don't need, data in the wrong format, and many records that are not useful. The data file is also very big. To make things easier we upload the data from the local machine and then generate a PKL file of the prepared dataframe.

In [2]:
filename = 'C:\\Users\\Abrah\\Desktop\\CST383 - Data Science\\Final Project\\Chess_Data.txt'
n = 3561470     #number of records in file
s = 300000      #desired sample size
skip = sorted(random.sample(range(n), n - s))

In [3]:
# column names, without the chess moves column
misc_columnNames = ['PNG_File_Pos - DELETE ME', 'Date', 'Game Result', 'W-ELO', 'B-ELO', 
                    'Num Moves', 'miscDate - DELETE ME', 'result - DELETE ME', 'wELO - DELETE ME', 'bELO - DELETE ME', 
                    'event date - DELETE ME', 'setup - DELETE ME', 'fen - DELETE ME', 'flag - DELETE ME', 'oyrange - DELETE ME', 
                    'bad len - DELETE ME']
delete_cols = ['PNG_File_Pos - DELETE ME', 'miscDate - DELETE ME', 'result - DELETE ME', 
               'wELO - DELETE ME', 'bELO - DELETE ME', 'event date - DELETE ME', 
               'setup - DELETE ME', 'fen - DELETE ME', 'flag - DELETE ME', 'oyrange - DELETE ME', 'bad len - DELETE ME']

In [4]:
# read all data except chess moves,
misc_chess_data = pd.read_csv(filename, comment = '#', infer_datetime_format = True, header = None, sep = ' ', on_bad_lines = 'skip', skiprows = skip)
misc_chess_data.drop(misc_chess_data.columns[16], axis = 1, inplace = True)
misc_chess_data.columns = misc_columnNames
misc_chess_data.drop(labels = ['PNG_File_Pos - DELETE ME', 'miscDate - DELETE ME', 'result - DELETE ME', 
               'wELO - DELETE ME', 'bELO - DELETE ME', 'event date - DELETE ME', 
               'setup - DELETE ME', 'fen - DELETE ME', 'flag - DELETE ME', 'oyrange - DELETE ME', 'bad len - DELETE ME'], axis = 1, inplace = True)


C:\Users\Abrah\AppData\Local\Temp\ipykernel_9188\3410824919.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  misc_chess_data = pd.read_csv(filename, comment = '#', infer_datetime_format = True, header = None, sep = ' ', on_bad_lines = 'skip', skiprows = skip)


In [7]:
# sanity check
misc_chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves
0,2000.05.24,1/2-1/2,2851,2748,52
1,2000.06.23,1/2-1/2,2851,2748,75
2,2000.06.19,1-0,2851,None,73
3,2000.06.19,1-0,2851,None,71
4,2000.03.14,1-0,2851,None,93


In the above cell we can see that the B-ELO has None values. We will fix that shortly. Next we prepare isolate the chess moves from the other data to make it easier to prepare this portion of the data. The dataframe game_moves will be merged with the previous dataframe after processsing.

In [6]:
# Isolate game moves from everything else.
game_moves = pd.read_csv(filename, engine = 'python', sep = '###', on_bad_lines = 'skip', header = None, skiprows = skip)

# drop first column of game moves (this is the misc chess data)
game_moves.drop(game_moves.columns[0], axis = 1, inplace = True)

In [7]:
game_moves.head()

,1
0,W1.e4 B1.c5 W2.Nf3 B2.Nc6 W3.Bb5 B3.g6 W4.Bxc...
1,W1.e4 B1.c5 W2.Nf3 B2.Nc6 W3.d4 B3.cxd4 W4.Nx...
2,W1.e4 B1.c5 W2.Nf3 B2.d6 W3.c3 B3.Nf6 W4.Be2 ...
3,W1.e4 B1.c5 W2.Nf3 B2.d6 W3.d4 B3.cxd4 W4.Nxd...
4,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bc4 B3.Bc5 W4.c3...


In [8]:
# split game moves df into columns for each move. 
game_moves = game_moves.iloc[:, 0].str.lstrip()
game_moves = game_moves.iloc[:].str.split(pat = ' ', expand = True)
game_moves.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599
0,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.Bb5,B3.g6,W4.Bxc6,B4.dxc6,W5.d3,B5.Bg7,W6.h3,B6.Nf6,W7.Nc3,B7.O-O,W8.Be3,B8.Qa5,W9.Qd2,B9.Rd8,W10.O-O,B10.Bd7,W11.Bh6,B11.Qc7,W12.Bxg7,B12.Kxg7,W13.Qe3,B13.b6,W14.Nh2,B14.Rf8,W15.f4,B15.Rad8,W16.Rae1,B16.Bc8,W17.f5,B17.e5,W18.Rf2,B18.Qd6,W19.Ref1,B19.h6,W20.b3,B20.Qd4,W21.Qe1,B21.b5,W22.Ne2,B22.Qd6,W23.Ng3,B23.c4,W24.dxc4,B24.bxc4,W25.Qa5,B25.cxb3,W26.axb3,B26.Rd7,W27.Nf3,B27.c5,W28.Qc3,B28.Re7,W29.Ra1,B29.Rfe8,W30.Ra5,B30.Rc7,W31.Qe3,B31.Qb6,W32.Ra4,B32.Bb7,W33.Qc3,B33.Nd7,W34.Ra1,B34.c4,W35.b4,B35.f6,W36.fxg6,B36.Kxg6,W37.Nh4+,B37.Kh7,W38.Qf3,B38.Qe6,W39.Rxa7,B39.Rg8,W40.Nhf5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,

In [9]:
game_moves.dtypes

0      object
1      object
2      object
3      object
4      object
        ...  
595    object
596    object
597    object
598    object
599    object
Length: 600, dtype: object

In [10]:
# chess moves need to be strings, so convert objects to strings.
game_moves = game_moves.astype('string', copy=True, errors='raise')
game_moves.dtypes

0      string
1      string
2      string
3      string
4      string
        ...  
595    string
596    string
597    string
598    string
599    string
Length: 600, dtype: object

In [11]:
# merge misc data and game moves into one df.
chess_data = pd.concat([misc_chess_data, game_moves], axis = 1)

In [12]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,...,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599
0,2000.06.19,1-0,2851,2191,79,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.Bb5,B3.g6,W4.Bxc6,B4.dxc6,W5.d3,B5.Bg7,W6.h3,B6.Nf6,W7.Nc3,B7.O-O,W8.Be3,B8.Qa5,W9.Qd2,B9.Rd8,W10.O-O,B10.Bd7,W11.Bh6,B11.Qc7,W12.Bxg7,B12.Kxg7,W13.Qe3,B13.b6,W14.Nh2,B14.Rf8,W15.f4,B15.Rad8,W16.Rae1,B16.Bc8,W17.f5,B17.e5,W18.Rf2,B18.Qd6,W19.Ref1,B19.h6,W20.b3,B20.Qd4,W21.Qe1,B21.b5,W22.Ne2,B22.Qd6,W23.Ng3,B23.c4,W24.dxc4,B24.bxc4,W25.Qa5,B25.cxb3,W26.axb3,B26.Rd7,W27.Nf3,B27.c5,W28.Qc3,B28.Re7,W29.Ra1,B29.Rfe8,W30.Ra5,B30.Rc7,W31.Qe3,B31.Qb6,W32.Ra4,B32.Bb7,W33.Qc3,B33.Nd7,W34.Ra1,B34.c4,W35.b4,B35.f6,W36.fxg6,B36.Kxg6,W37.Nh4+,B37.Kh7,W38.Qf3,B38.Qe6,W39.Rxa7,B39.Rg8,W40.Nhf5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA

The merging of the dataframes was successful. There are a few remaining steps for data preparation. We need to make sure we have the correct datatypes in place and also remove rows with missing data.

In [13]:
# remove rows with missing player ratings.
chess_data = chess_data[chess_data['B-ELO'] != 'None']
chess_data = chess_data[chess_data['W-ELO'] != 'None']

In [14]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,...,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599
0,2000.06.19,1-0,2851,2191,79,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.Bb5,B3.g6,W4.Bxc6,B4.dxc6,W5.d3,B5.Bg7,W6.h3,B6.Nf6,W7.Nc3,B7.O-O,W8.Be3,B8.Qa5,W9.Qd2,B9.Rd8,W10.O-O,B10.Bd7,W11.Bh6,B11.Qc7,W12.Bxg7,B12.Kxg7,W13.Qe3,B13.b6,W14.Nh2,B14.Rf8,W15.f4,B15.Rad8,W16.Rae1,B16.Bc8,W17.f5,B17.e5,W18.Rf2,B18.Qd6,W19.Ref1,B19.h6,W20.b3,B20.Qd4,W21.Qe1,B21.b5,W22.Ne2,B22.Qd6,W23.Ng3,B23.c4,W24.dxc4,B24.bxc4,W25.Qa5,B25.cxb3,W26.axb3,B26.Rd7,W27.Nf3,B27.c5,W28.Qc3,B28.Re7,W29.Ra1,B29.Rfe8,W30.Ra5,B30.Rc7,W31.Qe3,B31.Qb6,W32.Ra4,B32.Bb7,W33.Qc3,B33.Nd7,W34.Ra1,B34.c4,W35.b4,B35.f6,W36.fxg6,B36.Kxg6,W37.Nh4+,B37.Kh7,W38.Qf3,B38.Qe6,W39.Rxa7,B39.Rg8,W40.Nhf5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA

In [15]:
# remove rows where the number of game moves is 0.
chess_data = chess_data[chess_data['Num Moves'] != 0]

In [16]:
# remove rows where the number of game moves is over 75 for each player (75 * 2 players)
chess_data = chess_data[chess_data['Num Moves'] <= 150]

In [17]:
# we only want games with openings that we can analyze, so get all games that 
# have num moves at least 16 (8 move each side)
chess_data = chess_data[chess_data['Num Moves'] >= 16]

In [18]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,...,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599
0,2000.06.19,1-0,2851,2191,79,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.Bb5,B3.g6,W4.Bxc6,B4.dxc6,W5.d3,B5.Bg7,W6.h3,B6.Nf6,W7.Nc3,B7.O-O,W8.Be3,B8.Qa5,W9.Qd2,B9.Rd8,W10.O-O,B10.Bd7,W11.Bh6,B11.Qc7,W12.Bxg7,B12.Kxg7,W13.Qe3,B13.b6,W14.Nh2,B14.Rf8,W15.f4,B15.Rad8,W16.Rae1,B16.Bc8,W17.f5,B17.e5,W18.Rf2,B18.Qd6,W19.Ref1,B19.h6,W20.b3,B20.Qd4,W21.Qe1,B21.b5,W22.Ne2,B22.Qd6,W23.Ng3,B23.c4,W24.dxc4,B24.bxc4,W25.Qa5,B25.cxb3,W26.axb3,B26.Rd7,W27.Nf3,B27.c5,W28.Qc3,B28.Re7,W29.Ra1,B29.Rfe8,W30.Ra5,B30.Rc7,W31.Qe3,B31.Qb6,W32.Ra4,B32.Bb7,W33.Qc3,B33.Nd7,W34.Ra1,B34.c4,W35.b4,B35.f6,W36.fxg6,B36.Kxg6,W37.Nh4+,B37.Kh7,W38.Qf3,B38.Qe6,W39.Rxa7,B39.Rg8,W40.Nhf5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA

In [19]:
# we have too many columns because of all the NA values. Get dataframe up to col 155. That's 5 cols for misc data plus 150 cols
# for the chess moves (there will be a max of 150 moves)
chess_data = chess_data[chess_data.columns[:155]]

In [20]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
0,2000.06.19,1-0,2851,2191,79,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.Bb5,B3.g6,W4.Bxc6,B4.dxc6,W5.d3,B5.Bg7,W6.h3,B6.Nf6,W7.Nc3,B7.O-O,W8.Be3,B8.Qa5,W9.Qd2,B9.Rd8,W10.O-O,B10.Bd7,W11.Bh6,B11.Qc7,W12.Bxg7,B12.Kxg7,W13.Qe3,B13.b6,W14.Nh2,B14.Rf8,W15.f4,B15.Rad8,W16.Rae1,B16.Bc8,W17.f5,B17.e5,W18.Rf2,B18.Qd6,W19.Ref1,B19.h6,W20.b3,B20.Qd4,W21.Qe1,B21.b5,W22.Ne2,B22.Qd6,W23.Ng3,B23.c4,W24.dxc4,B24.bxc4,W25.Qa5,B25.cxb3,W26.axb3,B26.Rd7,W27.Nf3,B27.c5,W28.Qc3,B28.Re7,W29.Ra1,B29.Rfe8,W30.Ra5,B30.Rc7,W31.Qe3,B31.Qb6,W32.Ra4,B32.Bb7,W33.Qc3,B33.Nd7,W34.Ra1,B34.c4,W35.b4,B35.f6,W36.fxg6,B36.Kxg6,W37.Nh4+,B37.Kh7,W38.Qf3,B38.Qe6,W39.Rxa7,B39.Rg8,W40.Nhf5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000.01.28,1/2-1/2,2851,2725,68,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.d4,B3.cxd4,W4.Nxd4,B4.Nf6,W5.Nc3,B5.e5,W6.Ndb5,B6.d6,W7.Bg5,B7.a6,W8.Na3,B8.b5,W9.Bxf6,B9.gxf6,W10.Nd5,B10.f5,W11.c3,B11.Bg7,W12.exf5,B12.Bxf5,W13.Nc2,B13.Be6,W14.a4,B14.O-O,W15.axb5,B15.axb5,W16.Rxa8,B16.Qxa8,W17.Nce3,B17.Qb7,W18.g4,B18.Ne7,W19.Bg2,B19.Nxd5,W20.Bxd5,B20.Qe7,W21.h4,B21.Rc8,W22.g5,B22.Rc5,W23.Be4,B23.d5,W24.Nxd5,B24.Bxd5,W25.Bxd5,B25.e4,W26.Ke2,B26.Qe5,W27.Bb3,B27.Qc7,W28.Qg1,B28.b4,W29.g6,B29.hxg6,W30.Qxg6,B30.Kf8,W31.Qg3,B31.Qxg3,W32.fxg3,B32.bxc3,W33.bxc3,B33.Rxc3,W34.Bd5,B34.Rxg3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2000.01.30,1-0,2851,2658,87,W1.e4,B1.c5,W2.Nf3,B2.d6,W3.d4,B3.cxd4,W4.Nxd4,B4.Nf6,W5.Nc3,B5.a6,W6.Be3,B6.Ng4,W7.Bg5,B7.h6,W8.Bh4,B8.g5,W9.Bg3,B9.Bg7,W10.h3,B10.Nf6,W11.Bc4,B11.Qb6,W12.O-O,B12.O-O,W13.Nde2,B13.Qxb2,W14.Bb3,B14.Qa3,W15.f4,B15.Nc6,W16.Kh1,B16.Be6,W17.Qd3,B17.Rac8,W18.fxg5,B18.hxg5,W19.Nd5,B19.Rfe8,W20.Rad1,B20.Nb4,W21.Qf3,B21.Nbxd5,W22.exd5,B22.Bd7,W23.c3,B23.a5,W24.Qd3,B24.a4,W25.Bc2,B25.Qc5,W26.Rxf6,B26.exf6,W27.Qh7+,B27.Kf8,W28.Nd4,B28.Re5,W29.Bxe5,B29.fxe5,W30.Ne6+,B30.Bxe6,W31.dxe6,B31.Rc7,W32.Bxa4,B32.d5,W33.Qf5,B33.Qc4,W34.Bd7,B34.Qf4,W35.Qb1,B35.fxe6,W36.Bxe6,B36.Ke7,W37.Bxd5,B37.Rd7,W38.c4,B38.Qe3,W39.Qh7,B39.Kd8,W40.Rb1,B40.Qf4,W41.Be6,B41.Re7,W42.Bg4,B42.Rf7,W43.Qd3+,B43.Qd4,W44.Qg6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,2000.05.21,1-0,2851,2637,51,W1.e4,B1.e5,W2.Nf3,B2.Nf6,W3.Nxe5,B3.d6,W4.Nf3,B4.Nxe4,W5.d4,B5.d5,W6.Bd3,B6.Be7,W7.O-O,B7.Nc6,W8.c4,B8.Nb4,W9.Be2,B9.O-O,W10.Nc3,B10.Bf5,W11.a3,B11.Nxc3,W12.bxc3,B12.Nc6,W13.Re1,B13.Bf6,W14.Bf4,B14.Ne7,W15.Qb3,B15.b6,W16.cxd5,B16.Nxd5,W17.Be5,B17.Bg4,W18.Rad1,B18.Be7,W19.h3,B19.Bh5,W20.g4,B20.Bg6,W21.Bg3,B21.Nf6,W22.Ne5,B22.Ne4,W23.Bf3,B23.Nxg3,W24.Nc6,B24.Qd6,W25.Nxe7+,B25.Kh8,W26.Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA

In [21]:
# we only care about the year that the game took place. reformat the date col to reflect that
# try a string function on the df.
new_column = chess_data['Date'].str.slice(0, 4, 1)
chess_data['Date'] = new_column

In [22]:
chess_data.shape

(158252, 155)

In [23]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
0,2000,1-0,2851,2191,79,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.Bb5,B3.g6,W4.Bxc6,B4.dxc6,W5.d3,B5.Bg7,W6.h3,B6.Nf6,W7.Nc3,B7.O-O,W8.Be3,B8.Qa5,W9.Qd2,B9.Rd8,W10.O-O,B10.Bd7,W11.Bh6,B11.Qc7,W12.Bxg7,B12.Kxg7,W13.Qe3,B13.b6,W14.Nh2,B14.Rf8,W15.f4,B15.Rad8,W16.Rae1,B16.Bc8,W17.f5,B17.e5,W18.Rf2,B18.Qd6,W19.Ref1,B19.h6,W20.b3,B20.Qd4,W21.Qe1,B21.b5,W22.Ne2,B22.Qd6,W23.Ng3,B23.c4,W24.dxc4,B24.bxc4,W25.Qa5,B25.cxb3,W26.axb3,B26.Rd7,W27.Nf3,B27.c5,W28.Qc3,B28.Re7,W29.Ra1,B29.Rfe8,W30.Ra5,B30.Rc7,W31.Qe3,B31.Qb6,W32.Ra4,B32.Bb7,W33.Qc3,B33.Nd7,W34.Ra1,B34.c4,W35.b4,B35.f6,W36.fxg6,B36.Kxg6,W37.Nh4+,B37.Kh7,W38.Qf3,B38.Qe6,W39.Rxa7,B39.Rg8,W40.Nhf5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000,1/2-1/2,2851,2725,68,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.d4,B3.cxd4,W4.Nxd4,B4.Nf6,W5.Nc3,B5.e5,W6.Ndb5,B6.d6,W7.Bg5,B7.a6,W8.Na3,B8.b5,W9.Bxf6,B9.gxf6,W10.Nd5,B10.f5,W11.c3,B11.Bg7,W12.exf5,B12.Bxf5,W13.Nc2,B13.Be6,W14.a4,B14.O-O,W15.axb5,B15.axb5,W16.Rxa8,B16.Qxa8,W17.Nce3,B17.Qb7,W18.g4,B18.Ne7,W19.Bg2,B19.Nxd5,W20.Bxd5,B20.Qe7,W21.h4,B21.Rc8,W22.g5,B22.Rc5,W23.Be4,B23.d5,W24.Nxd5,B24.Bxd5,W25.Bxd5,B25.e4,W26.Ke2,B26.Qe5,W27.Bb3,B27.Qc7,W28.Qg1,B28.b4,W29.g6,B29.hxg6,W30.Qxg6,B30.Kf8,W31.Qg3,B31.Qxg3,W32.fxg3,B32.bxc3,W33.bxc3,B33.Rxc3,W34.Bd5,B34.Rxg3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2000,1-0,2851,2658,87,W1.e4,B1.c5,W2.Nf3,B2.d6,W3.d4,B3.cxd4,W4.Nxd4,B4.Nf6,W5.Nc3,B5.a6,W6.Be3,B6.Ng4,W7.Bg5,B7.h6,W8.Bh4,B8.g5,W9.Bg3,B9.Bg7,W10.h3,B10.Nf6,W11.Bc4,B11.Qb6,W12.O-O,B12.O-O,W13.Nde2,B13.Qxb2,W14.Bb3,B14.Qa3,W15.f4,B15.Nc6,W16.Kh1,B16.Be6,W17.Qd3,B17.Rac8,W18.fxg5,B18.hxg5,W19.Nd5,B19.Rfe8,W20.Rad1,B20.Nb4,W21.Qf3,B21.Nbxd5,W22.exd5,B22.Bd7,W23.c3,B23.a5,W24.Qd3,B24.a4,W25.Bc2,B25.Qc5,W26.Rxf6,B26.exf6,W27.Qh7+,B27.Kf8,W28.Nd4,B28.Re5,W29.Bxe5,B29.fxe5,W30.Ne6+,B30.Bxe6,W31.dxe6,B31.Rc7,W32.Bxa4,B32.d5,W33.Qf5,B33.Qc4,W34.Bd7,B34.Qf4,W35.Qb1,B35.fxe6,W36.Bxe6,B36.Ke7,W37.Bxd5,B37.Rd7,W38.c4,B38.Qe3,W39.Qh7,B39.Kd8,W40.Rb1,B40.Qf4,W41.Be6,B41.Re7,W42.Bg4,B42.Rf7,W43.Qd3+,B43.Qd4,W44.Qg6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,2000,1-0,2851,2637,51,W1.e4,B1.e5,W2.Nf3,B2.Nf6,W3.Nxe5,B3.d6,W4.Nf3,B4.Nxe4,W5.d4,B5.d5,W6.Bd3,B6.Be7,W7.O-O,B7.Nc6,W8.c4,B8.Nb4,W9.Be2,B9.O-O,W10.Nc3,B10.Bf5,W11.a3,B11.Nxc3,W12.bxc3,B12.Nc6,W13.Re1,B13.Bf6,W14.Bf4,B14.Ne7,W15.Qb3,B15.b6,W16.cxd5,B16.Nxd5,W17.Be5,B17.Bg4,W18.Rad1,B18.Be7,W19.h3,B19.Bh5,W20.g4,B20.Bg6,W21.Bg3,B21.Nf6,W22.Ne5,B22.Ne4,W23.Bf3,B23.Nxg3,W24.Nc6,B24.Qd6,W25.Nxe7+,B25.Kh8,W26.Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [24]:
# take out games without dates.
chess_data = chess_data[chess_data['Date'].str.isnumeric()]

In [25]:
chess_data.shape

(158252, 155)

In [26]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
0,2000,1-0,2851,2191,79,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.Bb5,B3.g6,W4.Bxc6,B4.dxc6,W5.d3,B5.Bg7,W6.h3,B6.Nf6,W7.Nc3,B7.O-O,W8.Be3,B8.Qa5,W9.Qd2,B9.Rd8,W10.O-O,B10.Bd7,W11.Bh6,B11.Qc7,W12.Bxg7,B12.Kxg7,W13.Qe3,B13.b6,W14.Nh2,B14.Rf8,W15.f4,B15.Rad8,W16.Rae1,B16.Bc8,W17.f5,B17.e5,W18.Rf2,B18.Qd6,W19.Ref1,B19.h6,W20.b3,B20.Qd4,W21.Qe1,B21.b5,W22.Ne2,B22.Qd6,W23.Ng3,B23.c4,W24.dxc4,B24.bxc4,W25.Qa5,B25.cxb3,W26.axb3,B26.Rd7,W27.Nf3,B27.c5,W28.Qc3,B28.Re7,W29.Ra1,B29.Rfe8,W30.Ra5,B30.Rc7,W31.Qe3,B31.Qb6,W32.Ra4,B32.Bb7,W33.Qc3,B33.Nd7,W34.Ra1,B34.c4,W35.b4,B35.f6,W36.fxg6,B36.Kxg6,W37.Nh4+,B37.Kh7,W38.Qf3,B38.Qe6,W39.Rxa7,B39.Rg8,W40.Nhf5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000,1/2-1/2,2851,2725,68,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.d4,B3.cxd4,W4.Nxd4,B4.Nf6,W5.Nc3,B5.e5,W6.Ndb5,B6.d6,W7.Bg5,B7.a6,W8.Na3,B8.b5,W9.Bxf6,B9.gxf6,W10.Nd5,B10.f5,W11.c3,B11.Bg7,W12.exf5,B12.Bxf5,W13.Nc2,B13.Be6,W14.a4,B14.O-O,W15.axb5,B15.axb5,W16.Rxa8,B16.Qxa8,W17.Nce3,B17.Qb7,W18.g4,B18.Ne7,W19.Bg2,B19.Nxd5,W20.Bxd5,B20.Qe7,W21.h4,B21.Rc8,W22.g5,B22.Rc5,W23.Be4,B23.d5,W24.Nxd5,B24.Bxd5,W25.Bxd5,B25.e4,W26.Ke2,B26.Qe5,W27.Bb3,B27.Qc7,W28.Qg1,B28.b4,W29.g6,B29.hxg6,W30.Qxg6,B30.Kf8,W31.Qg3,B31.Qxg3,W32.fxg3,B32.bxc3,W33.bxc3,B33.Rxc3,W34.Bd5,B34.Rxg3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2000,1-0,2851,2658,87,W1.e4,B1.c5,W2.Nf3,B2.d6,W3.d4,B3.cxd4,W4.Nxd4,B4.Nf6,W5.Nc3,B5.a6,W6.Be3,B6.Ng4,W7.Bg5,B7.h6,W8.Bh4,B8.g5,W9.Bg3,B9.Bg7,W10.h3,B10.Nf6,W11.Bc4,B11.Qb6,W12.O-O,B12.O-O,W13.Nde2,B13.Qxb2,W14.Bb3,B14.Qa3,W15.f4,B15.Nc6,W16.Kh1,B16.Be6,W17.Qd3,B17.Rac8,W18.fxg5,B18.hxg5,W19.Nd5,B19.Rfe8,W20.Rad1,B20.Nb4,W21.Qf3,B21.Nbxd5,W22.exd5,B22.Bd7,W23.c3,B23.a5,W24.Qd3,B24.a4,W25.Bc2,B25.Qc5,W26.Rxf6,B26.exf6,W27.Qh7+,B27.Kf8,W28.Nd4,B28.Re5,W29.Bxe5,B29.fxe5,W30.Ne6+,B30.Bxe6,W31.dxe6,B31.Rc7,W32.Bxa4,B32.d5,W33.Qf5,B33.Qc4,W34.Bd7,B34.Qf4,W35.Qb1,B35.fxe6,W36.Bxe6,B36.Ke7,W37.Bxd5,B37.Rd7,W38.c4,B38.Qe3,W39.Qh7,B39.Kd8,W40.Rb1,B40.Qf4,W41.Be6,B41.Re7,W42.Bg4,B42.Rf7,W43.Qd3+,B43.Qd4,W44.Qg6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,2000,1-0,2851,2637,51,W1.e4,B1.e5,W2.Nf3,B2.Nf6,W3.Nxe5,B3.d6,W4.Nf3,B4.Nxe4,W5.d4,B5.d5,W6.Bd3,B6.Be7,W7.O-O,B7.Nc6,W8.c4,B8.Nb4,W9.Be2,B9.O-O,W10.Nc3,B10.Bf5,W11.a3,B11.Nxc3,W12.bxc3,B12.Nc6,W13.Re1,B13.Bf6,W14.Bf4,B14.Ne7,W15.Qb3,B15.b6,W16.cxd5,B16.Nxd5,W17.Be5,B17.Bg4,W18.Rad1,B18.Be7,W19.h3,B19.Bh5,W20.g4,B20.Bg6,W21.Bg3,B21.Nf6,W22.Ne5,B22.Ne4,W23.Bf3,B23.Nxg3,W24.Nc6,B24.Qd6,W25.Nxe7+,B25.Kh8,W26.Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [27]:
# we want the date to be a number.
chess_data = chess_data.astype({'Date': 'int'}, copy=True, errors='raise')

In [28]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
0,2000,1-0,2851,2191,79,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.Bb5,B3.g6,W4.Bxc6,B4.dxc6,W5.d3,B5.Bg7,W6.h3,B6.Nf6,W7.Nc3,B7.O-O,W8.Be3,B8.Qa5,W9.Qd2,B9.Rd8,W10.O-O,B10.Bd7,W11.Bh6,B11.Qc7,W12.Bxg7,B12.Kxg7,W13.Qe3,B13.b6,W14.Nh2,B14.Rf8,W15.f4,B15.Rad8,W16.Rae1,B16.Bc8,W17.f5,B17.e5,W18.Rf2,B18.Qd6,W19.Ref1,B19.h6,W20.b3,B20.Qd4,W21.Qe1,B21.b5,W22.Ne2,B22.Qd6,W23.Ng3,B23.c4,W24.dxc4,B24.bxc4,W25.Qa5,B25.cxb3,W26.axb3,B26.Rd7,W27.Nf3,B27.c5,W28.Qc3,B28.Re7,W29.Ra1,B29.Rfe8,W30.Ra5,B30.Rc7,W31.Qe3,B31.Qb6,W32.Ra4,B32.Bb7,W33.Qc3,B33.Nd7,W34.Ra1,B34.c4,W35.b4,B35.f6,W36.fxg6,B36.Kxg6,W37.Nh4+,B37.Kh7,W38.Qf3,B38.Qe6,W39.Rxa7,B39.Rg8,W40.Nhf5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000,1/2-1/2,2851,2725,68,W1.e4,B1.c5,W2.Nf3,B2.Nc6,W3.d4,B3.cxd4,W4.Nxd4,B4.Nf6,W5.Nc3,B5.e5,W6.Ndb5,B6.d6,W7.Bg5,B7.a6,W8.Na3,B8.b5,W9.Bxf6,B9.gxf6,W10.Nd5,B10.f5,W11.c3,B11.Bg7,W12.exf5,B12.Bxf5,W13.Nc2,B13.Be6,W14.a4,B14.O-O,W15.axb5,B15.axb5,W16.Rxa8,B16.Qxa8,W17.Nce3,B17.Qb7,W18.g4,B18.Ne7,W19.Bg2,B19.Nxd5,W20.Bxd5,B20.Qe7,W21.h4,B21.Rc8,W22.g5,B22.Rc5,W23.Be4,B23.d5,W24.Nxd5,B24.Bxd5,W25.Bxd5,B25.e4,W26.Ke2,B26.Qe5,W27.Bb3,B27.Qc7,W28.Qg1,B28.b4,W29.g6,B29.hxg6,W30.Qxg6,B30.Kf8,W31.Qg3,B31.Qxg3,W32.fxg3,B32.bxc3,W33.bxc3,B33.Rxc3,W34.Bd5,B34.Rxg3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2000,1-0,2851,2658,87,W1.e4,B1.c5,W2.Nf3,B2.d6,W3.d4,B3.cxd4,W4.Nxd4,B4.Nf6,W5.Nc3,B5.a6,W6.Be3,B6.Ng4,W7.Bg5,B7.h6,W8.Bh4,B8.g5,W9.Bg3,B9.Bg7,W10.h3,B10.Nf6,W11.Bc4,B11.Qb6,W12.O-O,B12.O-O,W13.Nde2,B13.Qxb2,W14.Bb3,B14.Qa3,W15.f4,B15.Nc6,W16.Kh1,B16.Be6,W17.Qd3,B17.Rac8,W18.fxg5,B18.hxg5,W19.Nd5,B19.Rfe8,W20.Rad1,B20.Nb4,W21.Qf3,B21.Nbxd5,W22.exd5,B22.Bd7,W23.c3,B23.a5,W24.Qd3,B24.a4,W25.Bc2,B25.Qc5,W26.Rxf6,B26.exf6,W27.Qh7+,B27.Kf8,W28.Nd4,B28.Re5,W29.Bxe5,B29.fxe5,W30.Ne6+,B30.Bxe6,W31.dxe6,B31.Rc7,W32.Bxa4,B32.d5,W33.Qf5,B33.Qc4,W34.Bd7,B34.Qf4,W35.Qb1,B35.fxe6,W36.Bxe6,B36.Ke7,W37.Bxd5,B37.Rd7,W38.c4,B38.Qe3,W39.Qh7,B39.Kd8,W40.Rb1,B40.Qf4,W41.Be6,B41.Re7,W42.Bg4,B42.Rf7,W43.Qd3+,B43.Qd4,W44.Qg6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,2000,1-0,2851,2637,51,W1.e4,B1.e5,W2.Nf3,B2.Nf6,W3.Nxe5,B3.d6,W4.Nf3,B4.Nxe4,W5.d4,B5.d5,W6.Bd3,B6.Be7,W7.O-O,B7.Nc6,W8.c4,B8.Nb4,W9.Be2,B9.O-O,W10.Nc3,B10.Bf5,W11.a3,B11.Nxc3,W12.bxc3,B12.Nc6,W13.Re1,B13.Bf6,W14.Bf4,B14.Ne7,W15.Qb3,B15.b6,W16.cxd5,B16.Nxd5,W17.Be5,B17.Bg4,W18.Rad1,B18.Be7,W19.h3,B19.Bh5,W20.g4,B20.Bg6,W21.Bg3,B21.Nf6,W22.Ne5,B22.Ne4,W23.Bf3,B23.Nxg3,W24.Nc6,B24.Qd6,W25.Nxe7+,B25.Kh8,W26.Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [29]:
chess_data.dtypes

Date            int32
Game Result    object
W-ELO          object
B-ELO          object
Num Moves       int64
                ...  
145            string
146            string
147            string
148            string
149            string
Length: 155, dtype: object

In [30]:
# convert Num Moves to 32 bit int to reduce df size
chess_data = chess_data.astype({'Num Moves': 'int'}, copy=True, errors='raise')

In [31]:
# the dtypes for B-ELO and W-ELO need to be the same. So we convert W-ELO and B-ELO to int
chess_data = chess_data.astype({'B-ELO': 'int'}, copy=True, errors='raise')
chess_data = chess_data.astype({'W-ELO': 'int'}, copy=True, errors='raise')

In [32]:
# change game result to string
chess_data = chess_data.astype({'Game Result': 'string'}, copy=True, errors='raise')

In [33]:
chess_data.dtypes

Date            int32
Game Result    string
W-ELO           int32
B-ELO           int32
Num Moves       int32
                ...  
145            string
146            string
147            string
148            string
149            string
Length: 155, dtype: object

In [34]:
# This code will be used to rename the col names.
def createMovesList1(x): # Number of moves is <= 9
  size = x.size
  return x.str.slice(3,size,1)
def createMovesList2(x): # Number of moves is <= 99
  size = x.size
  return x.str.slice(4,size,1)
def createMovesList3(x): # Number of moves is <= 100
  size = x.size
  return x.str.slice(5,size,1)

In [35]:
# using this code to rename the cols.
chess_data.iloc[:,5:23] = chess_data.iloc[:, 5:23].apply(createMovesList1)
chess_data.iloc[:,23:113] = chess_data.iloc[:, 23:113].apply(createMovesList2)
chess_data.iloc[:,113:] = chess_data.iloc[:, 113:].apply(createMovesList3)
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
0,2000,1-0,2851,2191,79,e4,c5,Nf3,Nc6,Bb5,g6,Bxc6,dxc6,d3,Bg7,h3,Nf6,Nc3,O-O,Be3,Qa5,Qd2,Rd8,O-O,Bd7,Bh6,Qc7,Bxg7,Kxg7,Qe3,b6,Nh2,Rf8,f4,Rad8,Rae1,Bc8,f5,e5,Rf2,Qd6,Ref1,h6,b3,Qd4,Qe1,b5,Ne2,Qd6,Ng3,c4,dxc4,bxc4,Qa5,cxb3,axb3,Rd7,Nf3,c5,Qc3,Re7,Ra1,Rfe8,Ra5,Rc7,Qe3,Qb6,Ra4,Bb7,Qc3,Nd7,Ra1,c4,b4,f6,fxg6,Kxg6,Nh4+,Kh7,Qf3,Qe6,Rxa7,Rg8,Nhf5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000,1/2-1/2,2851,2725,68,e4,c5,Nf3,Nc6,d4,cxd4,Nxd4,Nf6,Nc3,e5,Ndb5,d6,Bg5,a6,Na3,b5,Bxf6,gxf6,Nd5,f5,c3,Bg7,exf5,Bxf5,Nc2,Be6,a4,O-O,axb5,axb5,Rxa8,Qxa8,Nce3,Qb7,g4,Ne7,Bg2,Nxd5,Bxd5,Qe7,h4,Rc8,g5,Rc5,Be4,d5,Nxd5,Bxd5,Bxd5,e4,Ke2,Qe5,Bb3,Qc7,Qg1,b4,g6,hxg6,Qxg6,Kf8,Qg3,Qxg3,fxg3,bxc3,bxc3,Rxc3,Bd5,Rxg3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2000,1-0,2851,2658,87,e4,c5,Nf3,d6,d4,cxd4,Nxd4,Nf6,Nc3,a6,Be3,Ng4,Bg5,h6,Bh4,g5,Bg3,Bg7,h3,Nf6,Bc4,Qb6,O-O,O-O,Nde2,Qxb2,Bb3,Qa3,f4,Nc6,Kh1,Be6,Qd3,Rac8,fxg5,hxg5,Nd5,Rfe8,Rad1,Nb4,Qf3,Nbxd5,exd5,Bd7,c3,a5,Qd3,a4,Bc2,Qc5,Rxf6,exf6,Qh7+,Kf8,Nd4,Re5,Bxe5,fxe5,Ne6+,Bxe6,dxe6,Rc7,Bxa4,d5,Qf5,Qc4,Bd7,Qf4,Qb1,fxe6,Bxe6,Ke7,Bxd5,Rd7,c4,Qe3,Qh7,Kd8,Rb1,Qf4,Be6,Re7,Bg4,Rf7,Qd3+,Qd4,Qg6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,2000,1-0,2851,2637,51,e4,e5,Nf3,Nf6,Nxe5,d6,Nf3,Nxe4,d4,d5,Bd3,Be7,O-O,Nc6,c4,Nb4,Be2,O-O,Nc3,Bf5,a3,Nxc3,bxc3,Nc6,Re1,Bf6,Bf4,Ne7,Qb3,b6,cxd5,Nxd5,Be5,Bg4,Rad1,Be7,h3,Bh5,g4,Bg6,Bg3,Nf6,Ne5,Ne4,Bf3,Nxg3,Nc6,Qd6,Nxe7+,Kh8,Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,2000,1-0,2851,2023,45,d4,e6,g3,c5,Nf3,Nc6,Bg2,cxd4,Nxd4,a6,O-O,Qc7,c4,Nf6,Nc3,Nxd4,Qxd4,Bc5,Qd3,O-O,Bf4,d6,Rad1,Rd8,a3,Nh5,Bg5,f6,Bc1,f5,b4,Ba7,Bg5,Nf6,e4,fxe4,Nxe4,Nxe4,Bxe4,Rd7,Bxh7+,Kh8,Bg6,Qc6,Qe2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [36]:
# This function will be used to concatenate two strings from two arrays
def combineStrings(a1,a2):
  new_list = []
  for i in range(0, a1.size):
    new_list.append(a1[i] + a2[i])
  return new_list

In [37]:
# Creating a list of integer from 1 to 75
# Then covert list type from int to string
a = np.arange(1,76,1)
a_str = list(map(str,a))

In [38]:
# Create move piece names.
w_pieces = np.full(75, ['W'])
b_pieces = np.full(75, ['B'])
w_new_list = combineStrings(w_pieces, a_str)
b_new_list = combineStrings(b_pieces, a_str)

In [39]:
# Combine list but alternate between white and black piece.
values = [None] * (len(w_new_list) + len(b_new_list))
values[::2] = w_new_list
values[1::2] = b_new_list

In [40]:
# rename columns to show W1, B1, W2, B2, .....
keys = np.arange(0,150,1)
dictionary = dict(zip(keys,values))
chess_data.rename(columns = dictionary, inplace=True)
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,W1,B1,W2,B2,W3,B3,W4,B4,W5,B5,W6,B6,W7,B7,W8,B8,W9,B9,W10,B10,W11,B11,W12,B12,W13,B13,W14,B14,W15,B15,W16,B16,W17,B17,W18,B18,W19,B19,W20,B20,W21,B21,W22,B22,W23,B23,W24,B24,W25,B25,W26,B26,W27,B27,W28,B28,W29,B29,W30,B30,W31,B31,W32,B32,W33,B33,W34,B34,W35,B35,W36,B36,W37,B37,W38,B38,W39,B39,W40,B40,W41,B41,W42,B42,W43,B43,W44,B44,W45,B45,W46,B46,W47,B47,W48,B48,W49,B49,W50,B50,W51,B51,W52,B52,W53,B53,W54,B54,W55,B55,W56,B56,W57,B57,W58,B58,W59,B59,W60,B60,W61,B61,W62,B62,W63,B63,W64,B64,W65,B65,W66,B66,W67,B67,W68,B68,W69,B69,W70,B70,W71,B71,W72,B72,W73,B73,W74,B74,W75,B75
0,2000,1-0,2851,2191,79,e4,c5,Nf3,Nc6,Bb5,g6,Bxc6,dxc6,d3,Bg7,h3,Nf6,Nc3,O-O,Be3,Qa5,Qd2,Rd8,O-O,Bd7,Bh6,Qc7,Bxg7,Kxg7,Qe3,b6,Nh2,Rf8,f4,Rad8,Rae1,Bc8,f5,e5,Rf2,Qd6,Ref1,h6,b3,Qd4,Qe1,b5,Ne2,Qd6,Ng3,c4,dxc4,bxc4,Qa5,cxb3,axb3,Rd7,Nf3,c5,Qc3,Re7,Ra1,Rfe8,Ra5,Rc7,Qe3,Qb6,Ra4,Bb7,Qc3,Nd7,Ra1,c4,b4,f6,fxg6,Kxg6,Nh4+,Kh7,Qf3,Qe6,Rxa7,Rg8,Nhf5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000,1/2-1/2,2851,2725,68,e4,c5,Nf3,Nc6,d4,cxd4,Nxd4,Nf6,Nc3,e5,Ndb5,d6,Bg5,a6,Na3,b5,Bxf6,gxf6,Nd5,f5,c3,Bg7,exf5,Bxf5,Nc2,Be6,a4,O-O,axb5,axb5,Rxa8,Qxa8,Nce3,Qb7,g4,Ne7,Bg2,Nxd5,Bxd5,Qe7,h4,Rc8,g5,Rc5,Be4,d5,Nxd5,Bxd5,Bxd5,e4,Ke2,Qe5,Bb3,Qc7,Qg1,b4,g6,hxg6,Qxg6,Kf8,Qg3,Qxg3,fxg3,bxc3,bxc3,Rxc3,Bd5,Rxg3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2000,1-0,2851,2658,87,e4,c5,Nf3,d6,d4,cxd4,Nxd4,Nf6,Nc3,a6,Be3,Ng4,Bg5,h6,Bh4,g5,Bg3,Bg7,h3,Nf6,Bc4,Qb6,O-O,O-O,Nde2,Qxb2,Bb3,Qa3,f4,Nc6,Kh1,Be6,Qd3,Rac8,fxg5,hxg5,Nd5,Rfe8,Rad1,Nb4,Qf3,Nbxd5,exd5,Bd7,c3,a5,Qd3,a4,Bc2,Qc5,Rxf6,exf6,Qh7+,Kf8,Nd4,Re5,Bxe5,fxe5,Ne6+,Bxe6,dxe6,Rc7,Bxa4,d5,Qf5,Qc4,Bd7,Qf4,Qb1,fxe6,Bxe6,Ke7,Bxd5,Rd7,c4,Qe3,Qh7,Kd8,Rb1,Qf4,Be6,Re7,Bg4,Rf7,Qd3+,Qd4,Qg6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,2000,1-0,2851,2637,51,e4,e5,Nf3,Nf6,Nxe5,d6,Nf3,Nxe4,d4,d5,Bd3,Be7,O-O,Nc6,c4,Nb4,Be2,O-O,Nc3,Bf5,a3,Nxc3,bxc3,Nc6,Re1,Bf6,Bf4,Ne7,Qb3,b6,cxd5,Nxd5,Be5,Bg4,Rad1,Be7,h3,Bh5,g4,Bg6,Bg3,Nf6,Ne5,Ne4,Bf3,Nxg3,Nc6,Qd6,Nxe7+,Kh8,Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,2000,1-0,2851,2023,45,d4,e6,g3,c5,Nf3,Nc6,Bg2,cxd4,Nxd4,a6,O-O,Qc7,c4,Nf6,Nc3,Nxd4,Qxd4,Bc5,Qd3,O-O,Bf4,d6,Rad1,Rd8,a3,Nh5,Bg5,f6,Bc1,f5,b4,Ba7,Bg5,Nf6,e4,fxe4,Nxe4,Nxe4,Bxe4,Rd7,Bxh7+,Kh8,Bg6,Qc6,Qe2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,

In [41]:
total_games = chess_data.index.values.size + 1

def createGames(total_games):
  new_list = []
  for i in range(1, total_games):
    new_list.append('Game' + str(i))
  return new_list

index_list = createGames(total_games)
chess_data.index = index_list

In [42]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,W1,B1,W2,B2,W3,B3,W4,B4,W5,B5,W6,B6,W7,B7,W8,B8,W9,B9,W10,B10,W11,B11,W12,B12,W13,B13,W14,B14,W15,B15,W16,B16,W17,B17,W18,B18,W19,B19,W20,B20,W21,B21,W22,B22,W23,B23,W24,B24,W25,B25,W26,B26,W27,B27,W28,B28,W29,B29,W30,B30,W31,B31,W32,B32,W33,B33,W34,B34,W35,B35,W36,B36,W37,B37,W38,B38,W39,B39,W40,B40,W41,B41,W42,B42,W43,B43,W44,B44,W45,B45,W46,B46,W47,B47,W48,B48,W49,B49,W50,B50,W51,B51,W52,B52,W53,B53,W54,B54,W55,B55,W56,B56,W57,B57,W58,B58,W59,B59,W60,B60,W61,B61,W62,B62,W63,B63,W64,B64,W65,B65,W66,B66,W67,B67,W68,B68,W69,B69,W70,B70,W71,B71,W72,B72,W73,B73,W74,B74,W75,B75
Game1,2000,1-0,2851,2191,79,e4,c5,Nf3,Nc6,Bb5,g6,Bxc6,dxc6,d3,Bg7,h3,Nf6,Nc3,O-O,Be3,Qa5,Qd2,Rd8,O-O,Bd7,Bh6,Qc7,Bxg7,Kxg7,Qe3,b6,Nh2,Rf8,f4,Rad8,Rae1,Bc8,f5,e5,Rf2,Qd6,Ref1,h6,b3,Qd4,Qe1,b5,Ne2,Qd6,Ng3,c4,dxc4,bxc4,Qa5,cxb3,axb3,Rd7,Nf3,c5,Qc3,Re7,Ra1,Rfe8,Ra5,Rc7,Qe3,Qb6,Ra4,Bb7,Qc3,Nd7,Ra1,c4,b4,f6,fxg6,Kxg6,Nh4+,Kh7,Qf3,Qe6,Rxa7,Rg8,Nhf5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game2,2000,1/2-1/2,2851,2725,68,e4,c5,Nf3,Nc6,d4,cxd4,Nxd4,Nf6,Nc3,e5,Ndb5,d6,Bg5,a6,Na3,b5,Bxf6,gxf6,Nd5,f5,c3,Bg7,exf5,Bxf5,Nc2,Be6,a4,O-O,axb5,axb5,Rxa8,Qxa8,Nce3,Qb7,g4,Ne7,Bg2,Nxd5,Bxd5,Qe7,h4,Rc8,g5,Rc5,Be4,d5,Nxd5,Bxd5,Bxd5,e4,Ke2,Qe5,Bb3,Qc7,Qg1,b4,g6,hxg6,Qxg6,Kf8,Qg3,Qxg3,fxg3,bxc3,bxc3,Rxc3,Bd5,Rxg3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game3,2000,1-0,2851,2658,87,e4,c5,Nf3,d6,d4,cxd4,Nxd4,Nf6,Nc3,a6,Be3,Ng4,Bg5,h6,Bh4,g5,Bg3,Bg7,h3,Nf6,Bc4,Qb6,O-O,O-O,Nde2,Qxb2,Bb3,Qa3,f4,Nc6,Kh1,Be6,Qd3,Rac8,fxg5,hxg5,Nd5,Rfe8,Rad1,Nb4,Qf3,Nbxd5,exd5,Bd7,c3,a5,Qd3,a4,Bc2,Qc5,Rxf6,exf6,Qh7+,Kf8,Nd4,Re5,Bxe5,fxe5,Ne6+,Bxe6,dxe6,Rc7,Bxa4,d5,Qf5,Qc4,Bd7,Qf4,Qb1,fxe6,Bxe6,Ke7,Bxd5,Rd7,c4,Qe3,Qh7,Kd8,Rb1,Qf4,Be6,Re7,Bg4,Rf7,Qd3+,Qd4,Qg6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game4,2000,1-0,2851,2637,51,e4,e5,Nf3,Nf6,Nxe5,d6,Nf3,Nxe4,d4,d5,Bd3,Be7,O-O,Nc6,c4,Nb4,Be2,O-O,Nc3,Bf5,a3,Nxc3,bxc3,Nc6,Re1,Bf6,Bf4,Ne7,Qb3,b6,cxd5,Nxd5,Be5,Bg4,Rad1,Be7,h3,Bh5,g4,Bg6,Bg3,Nf6,Ne5,Ne4,Bf3,Nxg3,Nc6,Qd6,Nxe7+,Kh8,Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game5,2000,1-0,2851,2023,45,d4,e6,g3,c5,Nf3,Nc6,Bg2,cxd4,Nxd4,a6,O-O,Qc7,c4,Nf6,Nc3,Nxd4,Qxd4,Bc5,Qd3,O-O,Bf4,d6,Rad1,Rd8,a3,Nh5,Bg5,f6,Bc1,f5,b4,Ba7,Bg5,Nf6,e4,fxe4,Nxe4,Nxe4,Bxe4,Rd7,Bxh7+,Kh8,Bg6,Qc6,Qe2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,

We finally have a dataset that has been prepared and formatted to our needs.

In [43]:
chess_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 158252 entries, Game1 to Game158252
Columns: 155 entries, Date to B75
dtypes: int32(4), string(151)
memory usage: 185.9+ MB


In [44]:
chess_data.head(100)

,Date,Game Result,W-ELO,B-ELO,Num Moves,W1,B1,W2,B2,W3,B3,W4,B4,W5,B5,W6,B6,W7,B7,W8,B8,W9,B9,W10,B10,W11,B11,W12,B12,W13,B13,W14,B14,W15,B15,W16,B16,W17,B17,W18,B18,W19,B19,W20,B20,W21,B21,W22,B22,W23,B23,W24,B24,W25,B25,W26,B26,W27,B27,W28,B28,W29,B29,W30,B30,W31,B31,W32,B32,W33,B33,W34,B34,W35,B35,W36,B36,W37,B37,W38,B38,W39,B39,W40,B40,W41,B41,W42,B42,W43,B43,W44,B44,W45,B45,W46,B46,W47,B47,W48,B48,W49,B49,W50,B50,W51,B51,W52,B52,W53,B53,W54,B54,W55,B55,W56,B56,W57,B57,W58,B58,W59,B59,W60,B60,W61,B61,W62,B62,W63,B63,W64,B64,W65,B65,W66,B66,W67,B67,W68,B68,W69,B69,W70,B70,W71,B71,W72,B72,W73,B73,W74,B74,W75,B75
Game1,2000,1-0,2851,2191,79,e4,c5,Nf3,Nc6,Bb5,g6,Bxc6,dxc6,d3,Bg7,h3,Nf6,Nc3,O-O,Be3,Qa5,Qd2,Rd8,O-O,Bd7,Bh6,Qc7,Bxg7,Kxg7,Qe3,b6,Nh2,Rf8,f4,Rad8,Rae1,Bc8,f5,e5,Rf2,Qd6,Ref1,h6,b3,Qd4,Qe1,b5,Ne2,Qd6,Ng3,c4,dxc4,bxc4,Qa5,cxb3,axb3,Rd7,Nf3,c5,Qc3,Re7,Ra1,Rfe8,Ra5,Rc7,Qe3,Qb6,Ra4,Bb7,Qc3,Nd7,Ra1,c4,b4,f6,fxg6,Kxg6,Nh4+,Kh7,Qf3,Qe6,Rxa7,Rg8,Nhf5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game2,2000,1/2-1/2,2851,2725,68,e4,c5,Nf3,Nc6,d4,cxd4,Nxd4,Nf6,Nc3,e5,Ndb5,d6,Bg5,a6,Na3,b5,Bxf6,gxf6,Nd5,f5,c3,Bg7,exf5,Bxf5,Nc2,Be6,a4,O-O,axb5,axb5,Rxa8,Qxa8,Nce3,Qb7,g4,Ne7,Bg2,Nxd5,Bxd5,Qe7,h4,Rc8,g5,Rc5,Be4,d5,Nxd5,Bxd5,Bxd5,e4,Ke2,Qe5,Bb3,Qc7,Qg1,b4,g6,hxg6,Qxg6,Kf8,Qg3,Qxg3,fxg3,bxc3,bxc3,Rxc3,Bd5,Rxg3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game3,2000,1-0,2851,2658,87,e4,c5,Nf3,d6,d4,cxd4,Nxd4,Nf6,Nc3,a6,Be3,Ng4,Bg5,h6,Bh4,g5,Bg3,Bg7,h3,Nf6,Bc4,Qb6,O-O,O-O,Nde2,Qxb2,Bb3,Qa3,f4,Nc6,Kh1,Be6,Qd3,Rac8,fxg5,hxg5,Nd5,Rfe8,Rad1,Nb4,Qf3,Nbxd5,exd5,Bd7,c3,a5,Qd3,a4,Bc2,Qc5,Rxf6,exf6,Qh7+,Kf8,Nd4,Re5,Bxe5,fxe5,Ne6+,Bxe6,dxe6,Rc7,Bxa4,d5,Qf5,Qc4,Bd7,Qf4,Qb1,fxe6,Bxe6,Ke7,Bxd5,Rd7,c4,Qe3,Qh7,Kd8,Rb1,Qf4,Be6,Re7,Bg4,Rf7,Qd3+,Qd4,Qg6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game4,2000,1-0,2851,2637,51,e4,e5,Nf3,Nf6,Nxe5,d6,Nf3,Nxe4,d4,d5,Bd3,Be7,O-O,Nc6,c4,Nb4,Be2,O-O,Nc3,Bf5,a3,Nxc3,bxc3,Nc6,Re1,Bf6,Bf4,Ne7,Qb3,b6,cxd5,Nxd5,Be5,Bg4,Rad1,Be7,h3,Bh5,g4,Bg6,Bg3,Nf6,Ne5,Ne4,Bf3,Nxg3,Nc6,Qd6,Nxe7+,Kh8,Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game5,2000,1-0,2851,2023,45,d4,e6,g3,c5,Nf3,Nc6,Bg2,cxd4,Nxd4,a6,O-O,Qc7,c4,Nf6,Nc3,Nxd4,Qxd4,Bc5,Qd3,O-O,Bf4,d6,Rad1,Rd8,a3,Nh5,Bg5,f6,Bc1,f5,b4,Ba7,Bg5,Nf6,e4,fxe4,Nxe4,Nxe4,Bxe4,Rd7,Bxh7+,Kh8,Bg6,Qc6,Qe2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,

In [45]:
# we will upload pkl file to github and use it for visualization
chess_data.to_pickle("./chess_data.pkl", compression = 'zip')